In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

In [ ]:
total_epochs = 100
batch_size = 100
learning_rate = 0.0002
n_latent = 128 # generator input으로 사용할 size of noise
n_hidden = 256
n_input = 28*28

In [ ]:
#Generator는 output으로 fake mnist image를 출력
with tf.variable_scope('generator'):
  G_W1 = tf.Variable(tf.random_normal([n_latent, n_hidden], stddev=0.01))
  G_b1 = tf.Variable(tf.zeros([n_hidden]))
  G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev=0.01))
  G_b2 = tf.Variable(tf.zeros([n_input]))

#Discriminator는 fake image와 real image를 구분하여 real image일 확률을 출력
with tf.variable_scope('discriminator'):
  D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev=0.01))
  D_b1 = tf.Variable(tf.zeros([n_hidden]))
  D_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev=0.01))
  D_b2 = tf.Variable(tf.zeros([1]))

In [ ]:
def random_noise(batch_size):
  return np.random.normal(size=[batch_size,n_latent])

def generator(X):
  hidden_layer = tf.nn.relu(tf.matmul(X,G_W1)+G_b1)
  fake_image = tf.nn.sigmoid(tf.matmul(hidden_layer,G_W2)+G_b2)
  return fake_image

def discriminator(X):
  hidden_layer = tf.nn.relu(tf.matmul(X,D_W1)+D_b1)
  predict = tf.nn.sigmoid(tf.matmul(hidden_layer,D_W2)+D_b1)
  return predict

In [ ]:
X = tf.placeholder(tf.float32, [None, n_input])
Z = tf.placeholder(tf.float32, [None, n_latent])

G_z = generator(Z)
D_fake = discriminator(G_z) # generator가 생성한 fake image에 대한 discriminator output
D_real = discriminator(X) # real image에 대한 discriminator output

#G는 Discriminator를 속여야 한다.
G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = D_fake, labels=tf.ones_like(D_fake)))

#D는 fake image에 대해서는 낮은 확률, real image에 대해서는 높은 확률을 출력해야 한다.
# Label * -log(logits) + (1 - Label) * -log(1 - logits)에 대하여 식을 넣어보자
# Label = 0 일 때는 fake image인 경우, logits이 0으로 가야 loss가 낮아진다.
# Label = 1 일 때는 real image인 경우, logits이 1로 가야 loss가 낮아진다.
D_fake_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = D_fake, labels = tf.zeros_like(D_fake)))
D_real_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = D_real, labels = tf.ones_like(D_real)))

D_loss = D_real_loss + D_fake_loss

tvar = tf.trainable_variables()
dvar = [var for var in tvar if 'discriminator' in var.name]
gvar = [var for var in tvar if 'generator' in var.name]

D_optim = tf.train.AdamOptimizer(learning_rate).minimize(D_loss, var_list=dvar)
G_optim = tf.train.AdamOptimizer(learning_rate).minimize(G_loss, var_list=gvar)

In [ ]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  total_batch = int(mnist.train.num_examples/batch_size)
  loss_val_D, loss_val_G = 0, 0

  for epoch in range(total_epochs):
    for i in range(total_batch):
      batch_X, _ = mnist.train.next_batch(batch_size)
      noise = random_noise(batch_size)

      _, loss_val_D = sess.run([D_optim, D_loss],
                             feed_dict={X:batch_X, Z:noise})
      _, loss_val_G = sess.run([G_optim, G_loss],
                             feed_dict={Z:noise})
    
      print("=======Epoch : ", epoch , " =======================================")
      print('D loss: {:.4}'.format(loss_val_D))
      print('G loss: {:.4}'.format(loss_val_G))

      if epoch == 0 or (epoch + 1) % 10 == 0:
        sample_size = 10
        noise = random_noise(sample_size)
        sample = sess.run(G_z, feed_dict={Z:noise})

        fig, ax = plt.subplots(1, sample_size, figsize=(sample_size, 1))

        for i in range(sample_size):
          ax[i].set_axis_off()
          ax[i].imshow(np.reshape(sample[i],(28,28)))

        plt.savefig('/gdrive/My Drive/Simple_GAN/{}.png'.format(str(epoch).zfill(3)),bbox_inches='tight')
        plt.close(fig)

print('Complete!')